<a href="https://colab.research.google.com/github/AgatElite/book-clustering-analysis/blob/main/Project_7_Stream_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>